## PDF2Markdown

- pymupdf4llm

In [ ]:
import pymupdf4llm
FILE_PATH = "/home/taejong_kim/workspace/rag-lab/database/BA_사용메뉴얼_분석가.pdf"
IMG_CONTENT_PATH = "/home/taejong_kim/workspace/rag-lab/database/image"

md_text = pymupdf4llm.to_markdown(
    FILE_PATH, write_images=True, image_path=IMG_CONTENT_PATH)

Processing /home/taejong_kim/workspace/rag-lab/database/BA_사용메뉴얼_분석가.pdf...
[                                        ] (0/27[                                        ] (  1/27[                                        ] (  2/27[                                        ] (  3/27[                                        ] (  4/27[                                        ] (  5/27[                                        ] (  6/275[=                                       ] (  7/2[=                                       ] (  8/2[=                                       ] (  9/275[=                                       ] ( 10/2[=                                       ] ( 11/2[=                                       ] ( 12/275[=                                       ] ( 13/27[==                                      ] ( 14/27[==                                      ] ( 15/27[==                                      ] ( 16/27[==                                      ] ( 17/27[==                        

In [ ]:
import re

splited_markdonw = [item for item in re.split(
    r'(?<!##)##(?!##)', md_text) if item != '']

In [ ]:
from langchain.schema import Document


def extract_images_and_create_documents(text):
    """
    Markdown 텍스트에서 ![]() 형식의 이미지를 추출하고, 
    본문과 이미지 리스트를 LangChain Document 객체로 변환
    """
    # 이미지 패턴 탐색 (예: ![alt text](image_url))
    image_pattern = re.compile(r'!\[.*?\]\((.*?)\)')

    # 이미지 리스트 추출
    images = image_pattern.findall(text)

    # 본문에서 이미지 태그 제거
    cleaned_text = image_pattern.sub('', text).strip()

    # LangChain Document 생성
    return Document(page_content=cleaned_text, metadata={"images": images})


docs4md = []
for md in splited_markdonw:
    doc = extract_images_and_create_documents(md)
    docs4md.append(doc)

## Vector Store


In [34]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import os
from dotenv import load_dotenv
import pickle

load_dotenv()

Python-dotenv could not parse statement starting at line 5


True

### HuggingFace
- model = 'jhgan/ko-sroberta-multitask'

In [ ]:
hf = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask')  # embedding model

vectorstore = FAISS.from_documents(docs4md, embedding=hf)  # vector store
# retriever = vectorstore.retriever()

In [ ]:
VECTORSTORE_PATH = "/home/taejong_kim/workspace/rag-lab/database/vector_store"
EMBEDDING_MODEL_PATH = "/home/taejong_kim/workspace/rag-lab/database/embedding_model" + \
    "/hf_embedding_model01.pkl"

vectorstore.save_local(VECTORSTORE_PATH)

with open(EMBEDDING_MODEL_PATH, "wb") as f:
    pickle.dump(hf, f)

In [24]:
vectorstore

In [ ]:
# 1. 저장된 임베딩 모델 불러오기

with open(EMBEDDING_MODEL_PATH, "rb") as f:
    embedding_model = pickle.load(f)

# 2. 저장된 벡터스토어 불러오기
vectorstore = FAISS.load_local(
    VECTORSTORE_PATH, embedding_model, allow_dangerous_deserialization=True)


# 3. 검색
query = "분석가"
results = vectorstore.similarity_search(query, top_k=5)

print('검색 결과: ', results[0].page_content)

검색 결과:  목록

1. BRIQUE Analytics에서 다중처리의 의미

2. 알고리즘 작성

3. 분석플로우 작성 및 실행


In [30]:
from operator import itemgetter

retriever = vectorstore.as_retriever()

itemgetter(query) | retriever

RunnableLambda(itemgetter('분석가'))
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fda8eb43860>, search_kwargs={})

In [29]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fda8eb43860>, search_kwargs={})

In [36]:
# VECTORSTORE_PATH = o

os.getenv("VECTORSTORE_PATH")

# VECTORSTORE_PATH

'/home/taejong_kim/workspace/rag-lab/database/vector_store'